Simulating etched particles with different etching speeds

In [1]:
import numpy as np
import tifffile
import os
from tqdm.notebook import tqdm

In [2]:
class simulated_etching:
    def __init__(self, px, V0, etching_rates, t_end, sim_size=(121, 151, 151)):
        self.px = px
        self.V0 = V0
        # self.R0 = (V0/np.pi*3/4)**(1/3)
        self.etching_rates = etching_rates
        self.t_end = t_end
        self.sim_size = sim_size

    def make_sphere(self, V):
        R = (V/np.pi*3/4)**(1/3)/self.px

        # Create coordinate grids
        z, y, x = np.indices(self.sim_size)
        
        # Compute center
        center = np.array(self.sim_size) / 2
        
        # Compute squared distances from center
        dist2 = (x - center[2])**2 + (y - center[1])**2 + (z - center[0])**2
        
        # Create boolean mask for the sphere
        sphere = dist2 <= R**2
        
        return sphere.astype(np.uint8)*255

    def lookup_etching_rates(self, t):
        # Define left edges (previous x)
        left_edges = np.concatenate(([0], self.etching_rates[:-1, 0]))
        right_edges = self.etching_rates[:, 0]
        
        # Find index where t falls
        i = np.searchsorted(right_edges, t+0.001)
        
        # Check if t is within range
        if i < self.etching_rates.shape[0] and left_edges[i] <= t < right_edges[i]:
            return self.etching_rates[i, 1]
        elif t >= right_edges[-1]:
            return self.etching_rates[-1, 1]
        else:
            # print([i, t, left_edges[i], right_edges[i]])
            return None  # or np.nan if you prefer numeric output
    
    def generate(self, out_dir, dt):
        os.makedirs(out_dir, exist_ok=True)
        t = 0
        V = self.V0
        pbar = tqdm(total=np.floor(self.t_end/dt),desc="Running", unit="iter")
        last_er = 0
        while t<self.t_end:
            model = self.make_sphere(V)
            tifffile.imwrite(f"{out_dir}/{int(t):04d} s.tif", model)
            er = self.lookup_etching_rates(t)
            if er is None:
                print("Boundary error at t = "+str(t))
                er = last_er
            dV = er*dt
            V -= dV
            t += dt
            last_er = er
            pbar.update(1)

In [3]:
#Volume data generated by Amira. See "Volume data for all NPs.csv". Volumetric etching rates calculated by linearly fitting volume vs. time. See "V_t_fitting". 
PdAu1 = simulated_etching(px=2, V0=939844, 
                          etching_rates=np.array([[433, 788], [725, 388], [1453, 561]]), t_end=1453) 
PdPt1 = simulated_etching(px=2*0.787, V0=187906, 
                          etching_rates=np.array([[778, 4.2], [1299, 28.3], [2986, 14.6]]), t_end=2986)
ChiralAu = simulated_etching(px=2*1.178, V0=7503083, 
                          etching_rates=np.array([[1614, 3405]]), t_end=1614)

In [5]:
# stage_t = [30, 60, 90, 120, 180, 240, 300, 360]
stage_t = [300, 360]

for st in stage_t:
    s_len = len(np.arange(-55, 60+5, 5))
    PdAu1.generate("simulated_outputs/PdAu1_"+str(st)+"s/", dt=st/s_len)
    
    s_len = len(np.arange(-60, 50+2.5, 2.5))
    PdPt1.generate("simulated_outputs/PdPt1_"+str(st)+"s/", dt=st/s_len)
    
    s_len = len(np.arange(-60, 55+2.5, 2.5))
    ChiralAu.generate("simulated_outputs/ChiralAu_"+str(st)+"s/", dt=st/s_len)

Running:   0%|          | 0/116.0 [00:00<?, ?iter/s]

Running:   0%|          | 0/447.0 [00:00<?, ?iter/s]

Running:   0%|          | 0/252.0 [00:00<?, ?iter/s]

Running:   0%|          | 0/96.0 [00:00<?, ?iter/s]

Running:   0%|          | 0/373.0 [00:00<?, ?iter/s]

Running:   0%|          | 0/210.0 [00:00<?, ?iter/s]

In [68]:
PdAu1 = simulated_etching(px=2, V0=939844, 
                          etching_rates=np.array([[433, 788], [725, 388], [1453, 561]]), t_end=1453)
PdAu3 = simulated_etching(px=2, V0=473966, 
                          etching_rates=np.array([[606, 347], [907, 277], [1285, 135]]), t_end=1285)
PdRu = simulated_etching(px=2*0.624, V0=200567, 
                          etching_rates=np.array([[313, 43], [927, 100], [1958, 35]]), t_end=1958)
Cu3As = simulated_etching(px=2*0.624, V0=88883, 
                          etching_rates=np.array([[960, 95]]), t_end=960)

In [67]:
s_len = len(np.arange(-55, 60, 5))
PdAu1.generate("simulated_outputs/PdAu1_exp/", dt=69/s_len)

Running:   0%|          | 0/484.0 [00:00<?, ?iter/s]

In [71]:
s_len = len(np.arange(-55, 60, 5))
PdAu3.generate("simulated_outputs/PdAu3_exp/", dt=71/s_len)

s_len = len(np.arange(-50, 55, 2.5))
PdRu.generate("simulated_outputs/PdRu_exp/", dt=98/s_len)

s_len = len(np.arange(-60, 50, 2.5))
Cu3As.generate("simulated_outputs/Cu3As_exp/", dt=137/s_len)

Running:   0%|          | 0/416.0 [00:00<?, ?iter/s]

Running:   0%|          | 0/839.0 [00:00<?, ?iter/s]

Running:   0%|          | 0/308.0 [00:00<?, ?iter/s]

/tmp/ipykernel_203264/151346614.py:11: RuntimeWarning: invalid value encountered in scalar power
  R = (V/np.pi*3/4)**(1/3)/self.px


In [72]:
s_len = len(np.arange(-55, 60, 5))
PdAu1.generate("simulated_outputs/PdAu1_ctrl/", dt=360/s_len)
PdAu3.generate("simulated_outputs/PdAu3_ctrl/", dt=360/s_len)

s_len = len(np.arange(-50, 55, 2.5))
PdRu.generate("simulated_outputs/PdRu_ctrl/", dt=360/s_len)

s_len = len(np.arange(-60, 50, 2.5))
Cu3As.generate("simulated_outputs/Cu3As_ctrl/", dt=360/s_len)

Running:   0%|          | 0/92.0 [00:00<?, ?iter/s]

Running:   0%|          | 0/82.0 [00:00<?, ?iter/s]

Running:   0%|          | 0/228.0 [00:00<?, ?iter/s]

Running:   0%|          | 0/117.0 [00:00<?, ?iter/s]

/tmp/ipykernel_203264/151346614.py:11: RuntimeWarning: invalid value encountered in scalar power
  R = (V/np.pi*3/4)**(1/3)/self.px
